# Filter on specific map key if value is null or key does not exist to add key-value

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;

//Map Expression Filter
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapReturnType;
import com.aerospike.client.exp.MapExp;
import com.aerospike.client.Value;
import com.aerospike.client.Value.NullValue;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

//Building map values
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.Value.MapValue;
import com.aerospike.client.Value;
import java.util.HashMap;
import java.util.Map;
import com.aerospike.client.cdt.CTX;

In [5]:
//Create Client Object, connect to the server
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

In [6]:
void addRecord(Integer keyIndex, String mapBinName, String mapKey1Val, String mapKey2Val, String subMapKey1Val, boolean addKey1){
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    WritePolicy wPolicy = new WritePolicy();
    wPolicy.sendKey = true;  //Optional, if you want to inspect the record key
    Key myRecKey = new Key("test", "testset", Value.get("key"+keyIndex));
    HashMap <String, Value> profileObj = new HashMap <String, Value>();
    if(addKey1) { profileObj.put("Key1", Value.get(mapKey1Val));  } 
    profileObj.put("Key2", Value.get(mapKey2Val));  
    
    //Sub-map (optional)
    HashMap <String, String> metaObj = new HashMap <String, String>();
    metaObj.put("Key1", subMapKey1Val);
    profileObj.put("subKey1", new MapValue(metaObj)); 
    

    client.operate(wPolicy, myRecKey, 
       MapOperation.put(mPolicy, mapBinName, Value.get("profile"), new MapValue(profileObj) )               
    );
    System.out.println("Rec added: [key "+keyIndex+"]: "+ client.get(null, myRecKey));
}

In [7]:
//Create test records
String binName = "myMapBin1";
addRecord(1, binName, "val11", "val12", "vals11", true);
addRecord(2, binName, "val21", "val22", "vals21", true);
addRecord(3, binName, "val31", "val32", "vals31", false);
addRecord(4, binName, null, "val42", "vals41", true);



Rec added: [key 1]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val12, Key1=val11, subKey1={Key1=vals11}}}))
Rec added: [key 2]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val22, Key1=val21, subKey1={Key1=vals21}}}))
Rec added: [key 3]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val32, subKey1={Key1=vals31}}}))
Rec added: [key 4]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val42, Key1=null, subKey1={Key1=vals41}}}))


In [8]:
void updateRecord(Integer keyIndex, String mapBinName, String mapKey1Val){
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    WritePolicy wPolicy = new WritePolicy();
    wPolicy.sendKey = true;  //Optional, if you want to inspect the record key
    Key myRecKey = new Key("test", "testset", Value.get("key"+keyIndex));
    
    Expression mapKeyFilter = Exp.build(
        Exp.eq(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.NIL, Exp.val("Key1"), Exp.mapBin(mapBinName), CTX.mapKey(Value.get("profile"))),
            Exp.nil() )
    );
 
    wPolicy.filterExp = mapKeyFilter;
    
    client.operate(wPolicy, myRecKey, 
       MapOperation.put(mPolicy, mapBinName, Value.get("Key1"), Value.get(mapKey1Val), CTX.mapKey(Value.get("profile")) )              
    );
    System.out.println("Read record: [key "+keyIndex+"]: "+ client.get(null, myRecKey));
}

In [9]:
//Update all records, should only update ones with Key1 missing or Key1 value null
String binName = "myMapBin1";
updateRecord(1, binName, "val11_updated");
updateRecord(2, binName, "val21_updated");
updateRecord(3, binName, "val31_updated");
updateRecord(4, binName, "val41_updated");



Read record: [key 1]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val12, Key1=val11, subKey1={Key1=vals11}}}))
Read record: [key 2]: (gen:1),(exp:0),(bins:(myMapBin1:{profile={Key2=val22, Key1=val21, subKey1={Key1=vals21}}}))
Read record: [key 3]: (gen:2),(exp:0),(bins:(myMapBin1:{profile={Key2=val32, Key1=val31_updated, subKey1={Key1=vals31}}}))
Read record: [key 4]: (gen:2),(exp:0),(bins:(myMapBin1:{profile={Key2=val42, Key1=val41_updated, subKey1={Key1=vals41}}}))


# Cleanup

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"